## Brain Tumour Detection model using XGBoost Classifier and Hyperparameter Optimization using Raytune:

In [1]:
#pip install ray[tune]

In [2]:
# Importing Ray:
import ray
print('ray %s' % ray.__version__)

ray 1.11.0


In [3]:
# Importing required Libraries:
import os
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

In [5]:
# Creating a Objective function:
def objective(config: dict):
    
    # Reading the dataset:
    df = pd.read_csv("/home/codebind/Project/HPO_models/Zernike_Moments_YN_250.csv", header=None)
    
    # Shuffling the whole dataset:
    df = df.sample(frac=1, random_state=0).reset_index(drop=True)
    
    # Converting categorical values of Target feature into numerical:
    df[289].replace(['YES','NO'], [1,0], inplace=True)
    
    # Independent and Dependent features:
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Train-Test spilt:
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(x_train, label = y_train)
    test_set = xgb.DMatrix(x_test, label = y_test)
    
    # Train the classifier, using the Tune callback
    xgb.train(
              config,
              train_set,
              evals = [(test_set, "eval")],
              verbose_eval = False,
              callbacks = [TuneReportCheckpointCallback(filename="model.xgb")]
              )

def get_best_model_checkpoint(analysis):
    best_bst = xgb.Booster()
    best_bst.load_model(os.path.join(analysis.best_checkpoint, "model.xgb"))
    accuracy = 1. - analysis.best_result["eval-error"]
    
    print(f"Best model parameters: {analysis.best_config}")
    print(f"Best model total accuracy: {accuracy:.4f}")

    return best_bst

def tune_xgboost():

    search_space = {
                    "objective": "binary:logistic",
                    "learning_rate": tune.loguniform(0.0001, 2.0),
                    "eval_metric": ["logloss", "error"],
                    "max_depth": tune.randint(1, 20),
                    "min_child_weight": tune.choice([1,2,3,4,5,6,7,8,9,10])
                    }

    # This will enable aggressive early stopping of bad trials.

    scheduler = ASHAScheduler(
                              max_t = 100,  # 10 training iterations
                              grace_period = 1,
                              reduction_factor = 2
                             )


    analysis = tune.run(
                        objective,
                        metric = "eval-logloss",
                        mode = "min",
                        # You can add "gpu": 0.1 to allocate GPUs
                        resources_per_trial = {"cpu": 1},
                        config = search_space,
                        num_samples = 100,
                        scheduler = scheduler
                       )

    return analysis

if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument(
                        "--server-address",
                        type=str,
                        default=None,
                        required=False,
                        help="The address of server to connect to if using Ray Client."
                        )
    
    args, _ = parser.parse_known_args()

    if args.server_address:
        
        import ray
        ray.init(f"ray://{args.server_address}")

    analysis = tune_xgboost()

    # Load the best model checkpoint.
    if args.server_address:
        # If connecting to a remote server with Ray Client, checkpoint loading
        # should be wrapped in a task so it will execute on the server.
        # We have to make sure it gets executed on the same node that
        # ``tune.run`` is called on.
        from ray.util.ml_utils.node import force_on_current_node
        remote_fn = force_on_current_node(ray.remote(get_best_model_checkpoint))
        best_bst = ray.get(remote_fn.remote(analysis))
    else:
        best_bst = get_best_model_checkpoint(analysis)

2022-04-08 15:09:51,940	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
/home/codebind/anaconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trial name,status,loc,learning_rate,max_depth,min_child_weight
objective_d6eff_00000,RUNNING,192.168.0.113:183433,0.539955,12,6
objective_d6eff_00001,PENDING,,0.000244866,19,5
objective_d6eff_00002,PENDING,,0.0274654,13,8
objective_d6eff_00003,PENDING,,0.0123843,8,6
objective_d6eff_00004,PENDING,,0.133902,1,9
objective_d6eff_00005,PENDING,,0.0935664,5,9
objective_d6eff_00006,PENDING,,0.00108695,1,3
objective_d6eff_00007,PENDING,,0.00136489,14,7
objective_d6eff_00008,PENDING,,0.465917,4,7
objective_d6eff_00009,PENDING,,0.000972954,19,1


Result for objective_d6eff_00000:
  date: 2022-04-08_15-09-55
  done: false
  eval-error: 0.32
  eval-logloss: 0.644859
  experiment_id: a7d7e7693b96486a84818e4bb90cc80b
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 183433
  should_checkpoint: true
  time_since_restore: 0.8238930702209473
  time_this_iter_s: 0.8238930702209473
  time_total_s: 0.8238930702209473
  timestamp: 1649410795
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6eff_00000
  


Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_d6eff_00000,RUNNING,192.168.0.113:183433,0.539955,12,6,5,3.43852,0.453552,0.22
objective_d6eff_00001,RUNNING,192.168.0.113:183429,0.000244866,19,5,,,,
objective_d6eff_00002,RUNNING,192.168.0.113:183427,0.0274654,13,8,,,,
objective_d6eff_00003,RUNNING,192.168.0.113:183434,0.0123843,8,6,,,,
objective_d6eff_00004,RUNNING,192.168.0.113:183430,0.133902,1,9,,,,
objective_d6eff_00005,RUNNING,192.168.0.113:183428,0.0935664,5,9,,,,
objective_d6eff_00006,RUNNING,192.168.0.113:183431,0.00108695,1,3,,,,
objective_d6eff_00007,RUNNING,192.168.0.113:183432,0.00136489,14,7,,,,
objective_d6eff_00008,PENDING,,0.465917,4,7,,,,
objective_d6eff_00009,PENDING,,0.000972954,19,1,,,,


Result for objective_d6eff_00004:
  date: 2022-04-08_15-09-59
  done: true
  eval-error: 0.4
  eval-logloss: 0.683707
  experiment_id: 4b4531f6d42a472a9ebde1530dfa4b58
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 183430
  should_checkpoint: true
  time_since_restore: 0.8566269874572754
  time_this_iter_s: 0.8566269874572754
  time_total_s: 0.8566269874572754
  timestamp: 1649410799
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6eff_00004
  
Result for objective_d6eff_00001:
  date: 2022-04-08_15-09-59
  done: true
  eval-error: 0.36
  eval-logloss: 0.693099
  experiment_id: d16f35a5dbe449798707a0464dda0e8f
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 183429
  should_checkpoint: true
  time_since_restore: 1.4861948490142822
  time_this_iter_s: 1.4861948490142822
  time_total_s: 1.4861948490142822
  timestamp: 1649410799
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6eff_00

...

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_d6eff_00095,RUNNING,192.168.0.113:187194,0.00383975,18,4,,,,
objective_d6eff_00096,RUNNING,192.168.0.113:187195,0.00882147,4,1,,,,
objective_d6eff_00097,RUNNING,192.168.0.113:187191,0.174819,1,7,,,,
objective_d6eff_00098,RUNNING,192.168.0.113:187320,1.29896,18,10,,,,
objective_d6eff_00099,PENDING,,0.0122484,19,3,,,,
objective_d6eff_00000,TERMINATED,192.168.0.113:183433,0.539955,12,6,10,8.82975,0.423817,0.22
objective_d6eff_00001,TERMINATED,192.168.0.113:183429,0.000244866,19,5,1,1.48619,0.693099,0.36
objective_d6eff_00002,TERMINATED,192.168.0.113:183427,0.0274654,13,8,2,2.88638,0.679991,0.32
objective_d6eff_00003,TERMINATED,192.168.0.113:183434,0.0123843,8,6,1,1.56002,0.690835,0.32
objective_d6eff_00004,TERMINATED,192.168.0.113:183430,0.133902,1,9,1,0.856627,0.683707,0.4


Result for objective_d6eff_00097:
  date: 2022-04-08_15-12-31
  done: false
  eval-error: 0.4
  eval-logloss: 0.681478
  experiment_id: 3cc40b847e56493fa071153f80665424
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 187191
  should_checkpoint: true
  time_since_restore: 0.7306008338928223
  time_this_iter_s: 0.7306008338928223
  time_total_s: 0.7306008338928223
  timestamp: 1649410951
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6eff_00097
  
Result for objective_d6eff_00095:
  date: 2022-04-08_15-12-31
  done: true
  eval-error: 0.4
  eval-logloss: 0.692269
  experiment_id: 4aabeda5c1db4a6b80e3fecbe83bcb7a
  hostname: abhishek
  iterations_since_restore: 1
  node_ip: 192.168.0.113
  pid: 187194
  should_checkpoint: true
  time_since_restore: 1.1539626121520996
  time_this_iter_s: 1.1539626121520996
  time_total_s: 1.1539626121520996
  timestamp: 1649410951
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6eff_00

Trial name,status,loc,learning_rate,max_depth,min_child_weight,iter,total time (s),eval-logloss,eval-error
objective_d6eff_00000,TERMINATED,192.168.0.113:183433,0.539955,12,6,10,8.82975,0.423817,0.22
objective_d6eff_00001,TERMINATED,192.168.0.113:183429,0.000244866,19,5,1,1.48619,0.693099,0.36
objective_d6eff_00002,TERMINATED,192.168.0.113:183427,0.0274654,13,8,2,2.88638,0.679991,0.32
objective_d6eff_00003,TERMINATED,192.168.0.113:183434,0.0123843,8,6,1,1.56002,0.690835,0.32
objective_d6eff_00004,TERMINATED,192.168.0.113:183430,0.133902,1,9,1,0.856627,0.683707,0.4
objective_d6eff_00005,TERMINATED,192.168.0.113:183428,0.0935664,5,9,2,2.39819,0.656704,0.38
objective_d6eff_00006,TERMINATED,192.168.0.113:183431,0.00108695,1,3,1,1.1067,0.693057,0.4
objective_d6eff_00007,TERMINATED,192.168.0.113:183432,0.00136489,14,7,1,1.52987,0.692817,0.32
objective_d6eff_00008,TERMINATED,192.168.0.113:183848,0.465917,4,7,4,5.70678,0.54856,0.24
objective_d6eff_00009,TERMINATED,192.168.0.113:183849,0.000972954,19,1,1,2.34855,0.69277,0.22


2022-04-08 15:12:34,851	INFO tune.py:639 -- Total run time: 162.92 seconds (161.76 seconds for the tuning loop).


Best model parameters: {'objective': 'binary:logistic', 'learning_rate': 0.3295933497390647, 'eval_metric': ['logloss', 'error'], 'max_depth': 12, 'min_child_weight': 4}
Best model total accuracy: 0.8000


In [6]:
# Reading the dataset:
df = pd.read_csv('/home/codebind/Project/HPO_models/Zernike_Moments_YN_250.csv', header=None)

In [7]:
# Shuffling the whole dataset:
df = df.sample(frac=1, random_state=0).reset_index(drop=True)

In [8]:
# Converting categorical values of Target feature into numerical:
df[289].replace(['YES','NO'], [1,0], inplace=True)

In [9]:
# independent and dependent features:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [10]:
# Train-Test spilt
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# Using Best parameters found from HyperParameter Optimization done using Optuna:
xgb = XGBClassifier(
                    learning_rate = 0.3295,
                    max_depth = 12,
                    min_child_weight = 4
                   )

In [13]:
# Fitting the Training data:
xgb.fit(x_train, y_train)

[15:13:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.3295,
              max_delta_step=0, max_depth=12, min_child_weight=4, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
# Predicting the test data:
y_pred = xgb.predict(x_test)

### Results:

In [15]:
print("Accuracy :", round(accuracy_score(y_test, y_pred), 2)*100, "%")

Accuracy : 84.0 %


In [16]:
print("Precision :", round(precision_score(y_test, y_pred), 2)*100, "%")

Precision : 87.0 %


In [17]:
print("Recall :", round(recall_score(y_test, y_pred), 2)*100, "%")

Recall : 87.0 %


In [18]:
print("f1_Score :", round(f1_score(y_test, y_pred), 2)*100, "%")

f1_Score : 87.0 %


In [19]:
print("Confusion Matrix :")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix :
[[15  4]
 [ 4 27]]


In [20]:
print("Classification Report :")
print(classification_report(y_test, y_pred))

Classification Report :
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        19
           1       0.87      0.87      0.87        31

    accuracy                           0.84        50
   macro avg       0.83      0.83      0.83        50
weighted avg       0.84      0.84      0.84        50

